### Importing libraries

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


## The tentative goal of this project will be to predict the hardest mutation mission

In [7]:
# Data obtained from the 'Starcraft 2 Co-Op Weekly Mutation Database' - 
# https://docs.google.com/spreadsheets/d/1NvYbNvHkivOKJ9vWf9EneXxvwMlCC4nkjqHlv6OCRQo/edit#gid=0

In [3]:
weekly_mutations = pd.read_csv('/Users/richardmason/Downloads/Starcraft 2 Co-Op Weekly Mutation Database - Weekly Mutations.csv')
available_mutators = pd.read_csv('/Users/richardmason/Downloads/Starcraft 2 Co-Op Weekly Mutation Database - Available Mutators.csv')

mutation_king = pd.read_csv('/Users/richardmason/Downloads/Starcraft 2 Co-Op Weekly Mutation Database - Mutation King.csv')
mutation_popularity = pd.read_csv('/Users/richardmason/Downloads/Starcraft 2 Co-Op Weekly Mutation Database - Mutation Popularity (Reddit).csv')
enemy_compositions = pd.read_csv('/Users/richardmason/Downloads/Starcraft 2 Co-Op Weekly Mutation Database - Enemy Compositions.csv')


## Going through each of the sheets and cleaning data

In [4]:
weekly_mutations.head()

,#,Week,Title [Mouse over for description],Map,Reddit,BNet,TL,Mutator 1,Mutator 2,Mutator 3,Approval Rating,Commander of the week,Difficulty Mean,Difficulty Mode
0,1.0,2016-05-16,Train of the Dead,Oblivion Express,[LINK],[LINK],[LINK],Walking Infested,Outbreak,Darkness,NaN,[Karax; Vorazun],2.64/5.00,Easy
1,2.0,2016-05-23,First Strike,Chain of Ascension,[LINK],[LINK],[LINK],Long Range,Short-sighted,Laser Drill,NaN,[VOTE HERE],[VOTE HERE],NaN
2,3.0,2016-05-30,Time Lock,Lock & Load,[LINK],[LINK],[LINK],Time Warp,Speed Freaks,Mag-nificent,NaN,[VOTE HERE],[VOTE HERE],NaN
3,4.0,2016-06-06,Bad Weather,Rifts to Korhal,[LINK],[LINK],[LINK],Void Rifts,Twister,NaN,NaN,[VOTE HERE],4.06/5.00,Very Difficult
4,5.0,2016-06-13,Whiteout,Void Launch,[LINK],[LINK],[LINK],Blizzard,Slim Pickings,NaN,NaN,[VOTE HERE],[VOTE HERE],NaN


In [11]:
# Dropping columns with [LINK], #, and dropping rows with missing difficulty
weekly_mutations.drop(columns = ['#', 'Reddit', 'BNet', 'TL'], inplace=True)

In [13]:
# dropping all the NaN's so that we can save 'approval rating' for a later analysis
weekly_mutations_approval = weekly_mutations.dropna()


In [16]:
weekly_mutations.drop(columns = 'Approval Rating', inplace=True)

In [17]:
weekly_mutations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 9 columns):
Week                                  126 non-null object
Title [Mouse over for description]    134 non-null object
Map                                   134 non-null object
Mutator 1                             133 non-null object
Mutator 2                             124 non-null object
Mutator 3                             79 non-null object
Commander of the week                 134 non-null object
Difficulty Mean                       134 non-null object
Difficulty Mode                       110 non-null object
dtypes: object(9)
memory usage: 9.8+ KB


In [18]:
weekly_mutations.dropna(inplace=True)

In [19]:
weekly_mutations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 123
Data columns (total 9 columns):
Week                                  66 non-null object
Title [Mouse over for description]    66 non-null object
Map                                   66 non-null object
Mutator 1                             66 non-null object
Mutator 2                             66 non-null object
Mutator 3                             66 non-null object
Commander of the week                 66 non-null object
Difficulty Mean                       66 non-null object
Difficulty Mode                       66 non-null object
dtypes: object(9)
memory usage: 5.2+ KB


In [34]:
weekly_mutations.columns

Index(['Week', 'Title [Mouse over for description]', 'Map', 'Mutator 1',
       'Mutator 2', 'Mutator 3', 'Commander of the week', 'Difficulty Mean',
       'Difficulty Mode'],
      dtype='object')

In [36]:
weekly_mutations.rename({'Title [Mouse over for description]': 'Title'}, axis = 1, inplace=True)

In [5]:
available_mutators

,Mutator,Type,Unnamed: 2
0,Afraid of the Dark,Player Impairment,Vision provided by all sources is extremely li...
1,Aggressive Deployment,Enemy Spawn,Additional enemy units are periodically deploy...
2,Alien Incubation,Enemy Spawn,All enemy units spawn Broodlings upon death.
3,Avenger,Enemy Buff,"Enemy units gain increased attack speed, armor..."
4,Barrier,Enemy Buff,Enemy units gain a temporary shield upon the f...
5,Black Death,Enemy Buff,Some enemy units carry a plague that deals dam...
6,Blizzard,Avoid the circle,"Storm clouds move across the map, damaging and..."
7,Chaos Studios,Other,Mutators are chosen at random and periodically...
8,Concussive Attacks,Enemy Buff,Player units are slowed by all enemy attacks.
9,Darkness,Player Impairment,Previously explored areas remain blacked out o...


In [21]:
available_mutators.columns

Index(['Mutator', 'Type', 'Unnamed: 2'], dtype='object')

In [26]:
# Renaming this column
available_mutators.rename({'Unnamed: 2': 'Description'}, axis = 1, inplace=True)

In [29]:
mutation_king.head() # This data is pretty useless to us- we won't use this at all

,Commander,Wins,Unnamed: 2,Mutation,Raynor,Kerrigan,Artanis,Swann,Zagara,Vorazun,Karax,Abathur,Alarak,Nova,Stukov,Fenix,Dehaka,Han and Horner,Tychus
0,Stukov,29.3,NaN,Train of the Dead,NaN,NaN,NaN,NaN,NaN,0.5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Karax,24.1,NaN,First Strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Vorazun,21.0,NaN,Time Lock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nova,11.2,NaN,Bad Weather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Raynor,10.6,NaN,Whiteout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
mutation_popularity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 5 columns):
#             123 non-null float64
Title         124 non-null object
Upvotes       123 non-null float64
Comments      123 non-null float64
Popularity    37 non-null object
dtypes: float64(3), object(2)
memory usage: 4.9+ KB


In [37]:
mutation_popularity['Popularity']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
       ... 
94      75%
95      21%
96      60%
97      15%
98      -4%
99      43%
100     73%
101     32%
102     18%
103     38%
104     -6%
105     48%
106     -8%
107     46%
108    -28%
109    -68%
110     37%
111     -8%
112      6%
113    -14%
114    -64%
115     27%
116     38%
117    -54%
118     38%
119     46%
120    -60%
121     67%
122     29%
123     29%
Name: Popularity, Length: 124, dtype: object

In [41]:
mutation_popularity.drop(columns = '#', inplace=True)

In [42]:
mutation_popularity_eda = mutation_popularity.dropna()

In [52]:
mutation_popularity_eda['Popularity'] = mutation_popularity_eda['Popularity'].str.replace('%', '')
mutation_popularity_eda['Popularity'] = mutation_popularity_eda['Popularity'].astype(float)

/Users/richardmason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/richardmason/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [53]:
mutation_popularity_eda.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 87 to 123
Data columns (total 4 columns):
Title         37 non-null object
Upvotes       37 non-null float64
Comments      37 non-null float64
Popularity    37 non-null float64
dtypes: float64(3), object(1)
memory usage: 1.4+ KB


In [56]:
# Maybe we can predict and "fill in" the missing popularity values by the number of upvotes and comments??

In [54]:
mutation_popularity_eda.corr()

,Upvotes,Comments,Popularity
Upvotes,1.00000,0.33931,0.16352
Comments,0.33931,1.00000,-0.15205
Popularity,0.16352,-0.15205,1.00000


In [57]:
# unfortunately, it appears we must drop the Popularity columns, as the correlation is not QUITE what
# we were hoping for

In [58]:
mutation_popularity.drop(columns = 'Popularity', inplace = True)

In [59]:
# Now, we'll try and join the weekly mutations dataframe and the mutation_popularity dataframes on 'Title'

In [60]:
weekly_mutations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 123
Data columns (total 9 columns):
Week                     66 non-null object
Title                    66 non-null object
Map                      66 non-null object
Mutator 1                66 non-null object
Mutator 2                66 non-null object
Mutator 3                66 non-null object
Commander of the week    66 non-null object
Difficulty Mean          66 non-null object
Difficulty Mode          66 non-null object
dtypes: object(9)
memory usage: 5.2+ KB


In [61]:
mutation_popularity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 3 columns):
Title       124 non-null object
Upvotes     123 non-null float64
Comments    123 non-null float64
dtypes: float64(2), object(1)
memory usage: 3.0+ KB


In [69]:
mutation_popularity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124 entries, 0 to 123
Data columns (total 3 columns):
Title       124 non-null object
Upvotes     123 non-null float64
Comments    123 non-null float64
dtypes: float64(2), object(1)
memory usage: 3.5+ KB


In [73]:
weekly_mutations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 123
Data columns (total 9 columns):
Week                     66 non-null object
Title                    66 non-null object
Map                      66 non-null object
Mutator 1                66 non-null object
Mutator 2                66 non-null object
Mutator 3                66 non-null object
Commander of the week    66 non-null object
Difficulty Mean          66 non-null object
Difficulty Mode          66 non-null object
dtypes: object(9)
memory usage: 5.2+ KB
